### RHLF - LLM Model

This model is based on Reinforcement Learning 

In [1]:
# importing the dependencies 

from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch

/Users/rajendran/Desktop/LLM/llm_workshop/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
# Sample patient question-answer pairs
patient_data = [
    {"question": "What are the symptoms of COVID-19?", "answer": "Common symptoms of COVID-19 include fever, cough, and shortness of breath."},
    {"question": "How is diabetes diagnosed?", "answer": "Diabetes is diagnosed through tests such as fasting blood sugar, oral glucose tolerance test, etc."},
    {"question": "What treatments are available for asthma?", "answer": "Treatments for asthma include bronchodilators, inhaled corticosteroids, etc."},
    {"question": "What are the risk factors for heart disease?", "answer": "Risk factors for heart disease include high blood pressure, high cholesterol, smoking, obesity, diabetes, and a sedentary lifestyle."},
    {"question": "How does chemotherapy work?", "answer": "Chemotherapy works by targeting and killing rapidly dividing cells, which includes cancer cells. It can be administered orally or intravenously."},
    {"question": "What are the symptoms of a stroke?", "answer": "Symptoms of a stroke include sudden numbness or weakness in the face, arm, or leg, confusion, trouble speaking or understanding, and difficulty walking."},
    {"question": "What is the function of the pancreas?", "answer": "The pancreas produces enzymes that aid in digestion and hormones such as insulin and glucagon that regulate blood sugar levels."},
    {"question": "What are the benefits of regular exercise?", "answer": "Regular exercise can improve cardiovascular health, strengthen muscles and bones, boost mood, reduce stress, and help maintain a healthy weight."},
    {"question": "What is the role of antioxidants in the body?", "answer": "Antioxidants help neutralize harmful free radicals in the body, reducing oxidative stress and lowering the risk of chronic diseases such as cancer and heart disease."},
    {"question": "How does insulin regulate blood sugar levels?", "answer": "Insulin facilitates the uptake of glucose by cells, reducing blood sugar levels. It also signals the liver to store excess glucose as glycogen."}
]


In [3]:
# Load pre-trained Sentence Transformer model


model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
df = pd.DataFrame(patient_data)

In [4]:
# Embed the answers


def calculate_similarity(query, embeddings):
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    return cos_scores

In [5]:
def input_text_model(input_text):
    input_sentence_processed = ''.join([i for i in input_text if not i.isdigit()])
    query = input_sentence_processed
    answer_embeddings = model.encode(df['question'], convert_to_tensor=True)
    query_similarity = calculate_similarity(query, answer_embeddings)
    top_n = 1
    top_indices = query_similarity.argsort().cpu().numpy()[-top_n:]
    similar_answers = df.loc[top_indices, 'answer'].tolist()
    return similar_answers[-1]

In [6]:
# Collect human feedback and update model
def update_model_with_feedback(input_text, human_feedback):
    # Incorporate human feedback into training data
    df.loc[len(df)] = {'question': input_text, 'answer': human_feedback}
    answer_embeddings = model.encode(df['question'], convert_to_tensor=True)
    input_sentence_processed = ''.join([i for i in input_text if not i.isdigit()])
    query = input_sentence_processed
    query_similarity = calculate_similarity(query, answer_embeddings)
    top_n = 1
    top_indices = query_similarity.argsort().cpu().numpy()[-top_n:]
    similar_answers = df.loc[top_indices, 'answer'].tolist()
    return similar_answers[-1]
    # Re-embed updated answers
    #answer_embeddings = model.encode(df['answer'], convert_to_tensor=True)

In [18]:
# Example usage with feedback loop
patient_question = "symptoms of diabetics?"
initial_answer = input_text_model(patient_question)
print("Initial Answer:", initial_answer)
# Assume human provides feedback
yes_no=input("Are you happy with the answer?")
if yes_no.lower() != "yes":
    human_feedback =input("Provide me the data to train")
    #human_feedback = "The answer provided was helpful, but it could include more details about preventive measures."
     
    updated_answer = update_model_with_feedback(patient_question, human_feedback)
    print("Updated Answer:", updated_answer)
else :
    print("Happy for guiding you") 

Initial Answer: Common symptoms of diabetes include frequent urination, increased thirst, extreme hunger, unexplained weight loss, fatigue, blurred vision, slow healing wounds, and tingling or numbness in the hands and feet. These symptoms may vary depending on the type of diabetes and individual characteristics, with some people experiencing no symptoms initially.
Happy for guiding you
